In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *
import scipy.sparse as sps
from tqdm import tqdm

Tensorflow is not available


c:\Users\francesco\miniconda3\envs\RecSysFramework\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)
data_train_val = data_train + data_val

In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

slim_el_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(data_train, verbose=False)
slim_el_recommender.fit(alpha=0.002085488964546896, 
                        topK=303, 
                        l1_ratio=0.0026626042226214576)#**slim_el_study.best_params)

#slimelastic [alpha: 0.002085488964546896, topK: 303, l1_ratio: 0.0026626042226214576]

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

rp3beta_recommender = RP3betaRecommender(data_train, verbose=False)
rp3beta_recommender.fit(topK=35, 
                        alpha=0.3598110596257423, 
                        beta=0.1558338653772352, 
                        implicit=True, 
                        normalize_similarity=True)#**rp3beta_study.best_params)

#p3beta [topK: 35, alpha: 0.3598110596257423, beta: 0.1558338653772352, implicit: True, normalize_similarity: True]

In [ ]:
candidate_recommender_study = optuna.create_study(
    study_name="slimel+p3beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = DifferentLossScoresHybridRecommender(data_train, slim_el_recommender, rp3beta_recommender)
candidate_recommender.fit(
        norm = 2, 
        alpha = 0.6182214281226774)#**candidate_recommender_study.best_params)

In [ ]:
n_users, n_items = data_train_val.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'

In [ ]:
training_dataframe

In [ ]:
cutoff = 50

for user_id in range(n_users):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff, remove_seen_flag=True)
    training_dataframe.loc[user_id, "ItemID"] = recommendations

In [ ]:
training_dataframe

In [ ]:
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe

In [ ]:
data_val_coo = sps.coo_matrix(data_val)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})
correct_recommendations

In [ ]:
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe

In [ ]:
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)
training_dataframe

In [ ]:
topPop = TopPop(data_train)
topPop.fit()

slim_el= MultiThreadSLIM_SLIMElasticNetRecommender(data_train, verbose=False)
slim_el.fit(alpha=0.002085488964546896, 
            topK=303, 
            l1_ratio=0.0026626042226214576)

rp3beta= RP3betaRecommender(data_train, verbose=False)
rp3beta.fit(topK=35, 
            alpha=0.3598110596257423, 
            beta=0.1558338653772352, 
            implicit=True, 
            normalize_similarity=True)

easer = EASE_R_Recommender(data_train, verbose=True)
easer.fit(topK=72, 
                l2_norm=36.47305040353163)

itemknn = ItemKNNCFRecommender(data_train)
itemknn.fit(topK=180, 
            similarity="cosine", 
            normalize=True, 
            feature_weighting="TF-IDF")

p3alpha = P3alphaRecommender(data_train)
p3alpha.fit(topK=61, 
            alpha=0.4121129634953655, 
            implicit=True, 
            normalize_similarity=True)

slimbpr = SLIM_BPR_Cython(data_train)
slimbpr.fit(epochs=411, 
            symmetric=True, 
            lambda_i=0.3776970357741607, 
            lambda_j=0.04722986062962217, 
            learning_rate=0.0003013846889808713, 
            topK=60, 
            sgd_mode="adagrad", 
            gamma=1.1938389128360214e-05, 
            beta_1=0.002818160651487956, 
            beta_2=3.307005474510783e-07)


other_algorithms = {
    "TopPop": topPop,
    "SlimEl": slim_el,
    "P3Beta": rp3beta,
    "Easer": easer,
    "ItemKNN": itemknn,
    "P3alpha": p3alpha,
    "SLIM_BPR": slimbpr,
}

In [ ]:
training_dataframe = training_dataframe.set_index('UserID')

for user_id in tqdm(range(n_users)):  
    for rec_label, rec_instance in other_algorithms.items():
        
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)

        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})
training_dataframe

In [ ]:
item_popularity = np.ediff1d(sps.csc_matrix(data_train).indptr)

training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
training_dataframe

In [ ]:
user_popularity = np.ediff1d(sps.csr_matrix(data_train).indptr)

training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
training_dataframe

In [ ]:
training_dataframe = training_dataframe.sort_values("UserID").reset_index()
training_dataframe.drop(columns = ['index'], inplace=True)
training_dataframe

In [ ]:
groups = training_dataframe.groupby("UserID").size().values
groups

In [ ]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [ ]:
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [ ]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        train_recs,
        train_labs,
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train, train_recs, XGB_model, training_dataframe, data_val)
    
    return ev_map

study.optimize(objective, n_trials=1000)

In [ ]:
from xgboost import XGBRanker

XGB_model = XGBRanker(
    **study.best_params,
    verbosity=0, # 2 if self.verbose else 0,
    enable_categorical=True,
)
XGB_model.fit(
    train_recs,
    train_labs,
    group=groups,
    verbose=False
)

In [ ]:
def submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val):
    
    toppoprecommender = TopPop(data_train_val)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            predict = train_recs[train_recs["UserID"] == user_id]
            user_ids = predict["UserID"].values
            item_ids = predict["ItemID"].values
            predictions = XGB_model.predict(predict)
            if not list(predictions):
                predictions = toppoprecommender.recommend(0, cutoff = 10)
                item_list = [itemmap[i] for i in predictions]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                continue
            predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions.ravel()})
            predictions = predictions.sort_values(by="Predictions", ascending=False)
            for i in range(10):
                if predictions["ItemID"].values[i] in data_train_val[user_id].indices:
                    predictions["Predictions"].values[i] = -10
            recommendations = np.array(predictions["ItemID"].values[:10])
            item_list = [itemmap[i] for i in recommendations]
            writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                

In [ ]:
submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val)